In [81]:
import json
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import yaml
from box import ConfigBox
from sklearn.metrics import f1_score
from tqdm import tqdm


### Set web serice URL

In [82]:
# local testing
API_URL = "http://0.0.0.0:8080" 
# testing deployed service
# API_URL = "https://<YOUR_APP_NAME>.fly.dev" 

API_URL

'http://0.0.0.0:8080'

### Load feature columns and data

In [83]:
def load_params(params_path):
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
        params = ConfigBox(params)
    return params


proj_path = Path(os.getcwd()).parent.absolute()
params = load_params(proj_path/'params.yaml')
feat_cols = params.base.feat_cols
targ_col = params.base.targ_col
feat_cols, targ_col

(BoxList(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']),
 'Exited')

In [84]:
df = pd.concat([pd.read_csv(proj_path/'data'/'raw'/f'Churn_Modelling_{country}.csv') for country in ['France', 'Spain']])
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
2,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
3,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
4,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0


### Test `/predict` endpoint by sending one sample request

In [85]:
my_obj = [{
    'CreditScore': 755,
    'Age': 40,
    'Tenure': 2,
    'Balance': 137430.82,
    'NumOfProducts': 2,
    'HasCrCard': 0,
    'IsActiveMember': 0,
    'EstimatedSalary': 176768.59,
    'Exited': 0
  }]

x = requests.post(API_URL + '/predict', json = my_obj)
if x.ok:
    probs = x.json()
else:
    x.raise_for_status()

probs

[0.94]

### Create a functions for calling `/predict` endpoint

In [86]:
def get_prob(obj):
    x = requests.post(API_URL + '/predict', json = obj)
    if x.ok:
        probs = x.json()
    else:
        x.raise_for_status()
    return probs

get_prob(my_obj)

[0.94]

### Check what would f1-scores look like if we send the data from the same geographies that the model was trained on 

In [87]:
import time


def send_sample_requests(df):
    f1_score_list = []
    for _ in tqdm(range(50)):
        df_sample = df.sample(n=60)
        y_true = df_sample[targ_col].values
        obj = df_sample[feat_cols].to_dict('records')
        probs = get_prob(obj)
        y_pred = np.array([prob < 0.5 for prob in probs]).astype(int)
        f1 = f1_score(y_true, y_pred)
        f1_score_list.append(f1)
        time.sleep(1)
    return f1_score_list
        

In [88]:
f1_score_list = send_sample_requests(df)
f1_score_list

100%|██████████| 50/50 [00:52<00:00,  1.04s/it]


[0.9285714285714286,
 0.8235294117647058,
 1.0,
 0.5,
 0.8235294117647058,
 0.923076923076923,
 0.8571428571428571,
 0.8,
 0.888888888888889,
 0.782608695652174,
 0.8695652173913044,
 0.7692307692307693,
 0.8571428571428571,
 1.0,
 0.6666666666666666,
 0.9090909090909091,
 1.0,
 1.0,
 0.7692307692307692,
 0.9473684210526316,
 0.9090909090909091,
 0.6666666666666665,
 0.8571428571428571,
 1.0,
 0.923076923076923,
 0.9090909090909091,
 0.88,
 1.0,
 0.888888888888889,
 0.9411764705882353,
 0.8750000000000001,
 0.888888888888889,
 0.8235294117647058,
 0.923076923076923,
 0.8571428571428571,
 0.5,
 0.9565217391304348,
 0.8235294117647058,
 0.9,
 0.9523809523809523,
 0.8235294117647058,
 0.8333333333333334,
 0.9473684210526316,
 1.0,
 0.9333333333333333,
 0.9333333333333333,
 0.8571428571428571,
 0.9,
 0.9090909090909091,
 0.9]

### What would f1-scores look like for input data from new geography?

In [89]:
df_germany =  pd.read_csv(proj_path/'data'/'more_data'/'Churn_Modelling_Germany.csv')
#time.sleep(20) # this is to create a pause between requests. Later it'll be easier to distinguish the two types if requests
f1_score_list = send_sample_requests(df_germany)
f1_score_list


100%|██████████| 50/50 [00:51<00:00,  1.04s/it]


[0.5833333333333334,
 0.4444444444444445,
 0.5806451612903225,
 0.5555555555555556,
 0.2608695652173913,
 0.6060606060606061,
 0.45161290322580644,
 0.5384615384615384,
 0.64,
 0.5,
 0.5,
 0.56,
 0.411764705882353,
 0.4666666666666666,
 0.56,
 0.4615384615384615,
 0.45454545454545453,
 0.32000000000000006,
 0.4,
 0.5217391304347826,
 0.1,
 0.4,
 0.72,
 0.5454545454545455,
 0.4827586206896552,
 0.35714285714285715,
 0.5128205128205128,
 0.39999999999999997,
 0.3,
 0.4166666666666667,
 0.48484848484848486,
 0.5142857142857142,
 0.6,
 0.48,
 0.5294117647058824,
 0.4,
 0.5555555555555556,
 0.5882352941176471,
 0.4444444444444445,
 0.42424242424242425,
 0.47619047619047616,
 0.5185185185185185,
 0.5641025641025641,
 0.48275862068965514,
 0.4615384615384615,
 0.5,
 0.56,
 0.37037037037037035,
 0.5185185185185185,
 0.631578947368421]

### ... as expected, the values are much lower.
### But, typically, we can't compute model metrics on production data right away 
### because ground truth labels (y_true) might not be available until much-much later


### All we can do is try and look at "proxy" metrics that measure statistical differences (distances) between train data and production data

In [90]:
x = requests.get(API_URL + '/drift_data')
if x.ok:
    data = x.json()
else:
    x.raise_for_status()

df_p_vals = pd.DataFrame(json.loads(data))
df_p_vals

,time,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,2023-03-17 20:48:47.939415,0.284814,0.626956,0.995394,9.966909e-01,0.251245,1.000000,1.000000,0.138672
1,2023-03-17 20:48:48.991555,0.687762,0.597774,0.576022,1.658254e-02,0.945641,0.800531,0.317147,0.102329
2,2023-03-17 20:48:50.074958,0.178172,0.256725,0.999997,9.993206e-01,0.995000,0.998416,0.608368,0.239617
3,2023-03-17 20:48:51.106194,0.948212,0.759597,0.798703,2.545852e-01,0.570102,0.998416,0.999978,0.131595
4,2023-03-17 20:48:52.168989,0.698371,0.876803,0.998446,9.937741e-01,1.000000,1.000000,1.000000,0.180131
...,...,...,...,...,...,...,...,...,...
95,2023-03-17 20:50:40.912270,0.804722,0.004129,0.750795,0.000000e+00,0.999949,1.000000,1.000000,0.476735
96,2023-03-17 20:50:41.973214,0.968219,0.698197,0.886815,0.000000e+00,0.999949,0.993182,1.000000,0.777053
97,2023-03-17 20:50:43.018895,0.971559,0.140811,0.801245,0.000000e+00,0.750037,1.000000,0.075475,0.367774
98,2023-03-17 20:50:44.050894,0.968372,0.267510,0.999174,6.822660e-16,0.750037,0.998416,0.999087,0.604717


### Values below the threshold (e.g. 0.05) indicate possible data drift

In [91]:
import plotly_express as px

fig = px.line(df_p_vals, x='time', y=feat_cols)
fig.add_hline(y=0.05, line_color='red')